코드 2-1

In [ ]:
!gdown https://drive.google.com/uc?id=1pO27YTida7yUIGaj3YxvPWa4gFZWhjhy

In [ ]:
!pip install -U ultralytics

In [ ]:
from google.colab import files
from pathlib import Path
import cv2, os, time, uuid
from IPython.display import HTML, display
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

video_path = "/content/FA-50.mp4"
print("입력 영상:", video_path)

# === 2) 모델 로드 (YOLO11n) ===
model = YOLO("yolo11n.pt")  # 자동 다운로드

# === 3) 추론 설정 ===
CONF = 0.25           # 신뢰도 임계값
DEVICE = 0            # GPU(0) 또는 'cpu'
SHOW_EVERY = 30       # N프레임마다 코랩 셀에 미리보기 표시
OUTPUT_PATH = f"annotated_{Path(video_path).stem}.mp4"

# === 4) OpenCV로 프레임 단위 추론 & 시각화 ===
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (w, h))

frame_idx = 0
t0 = time.time()
print("추론 시작...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 추론 (단일 프레임, numpy 입력)
    results = model.predict(
        source=frame,
        conf=CONF,
        device=DEVICE,
        verbose=False
    )

    # 시각화 프레임 얻기 (BGR)
    annotated = results[0].plot()

    # 파일로 기록
    writer.write(annotated)

    # N 프레임마다 미리보기 (너무 자주 띄우면 느려질 수 있음)
    if frame_idx % SHOW_EVERY == 0:
        print(f"preview frame #{frame_idx}")
        cv2_imshow(annotated)  # 코랩 전용 이미지 표시
    frame_idx += 1

cap.release()
writer.release()
elapsed = time.time() - t0
print(f"✅ 처리 완료: {OUTPUT_PATH}  (총 {frame_idx} 프레임, {elapsed:.1f}s)")

# === 5) 주석된 MP4를 셀 안에서 즉시 재생(HTML5 video) ===
# 코랩은 /content 경로의 파일을 직접 <video>로 재생할 수 있습니다.
video_id = str(uuid.uuid4())
display(HTML(f"""
<video id="{video_id}" width="720" controls>
  <source src="{OUTPUT_PATH}" type="video/mp4">
  브라우저가 video 태그를 지원하지 않습니다.
</video>
<script>
  // 자동 재생(무음) 시도: 실패해도 무시
  const v = document.getElementById("{video_id}");
  v.muted = true;
  v.play().catch(()=>{{}});
</script>
"""))

코드 2-2(2-1과 연계)

In [37]:
#객체 추적 라이브러리 설치
!pip install -q deep_sort_realtime opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.1 MB/s eta 0:00:00


In [ ]:
#객체 추적 실행 코드 (YOLOv8 + Deep SORT)
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Deep SORT 초기화
tracker = DeepSort(max_age=30)

# 비디오 읽기
cap = cv2.VideoCapture(video_path)

# 비디오 저장용 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('tracked_output.mp4', fourcc, 20.0, (
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 객체 탐지
    results = model(frame, verbose=False)[0]

    detections = []
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, int(cls)))

    # Deep SORT로 객체 추적
    tracks = tracker.update_tracks(detections, frame=frame)

    # 결과 시각화
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
from tqdm import tqdm  # 진행상황 출력용

# Deep SORT 초기화
tracker = DeepSort(max_age=30)

# 궤적 저장용
trajectories = defaultdict(list)

# 비디오 열기
cap = cv2.VideoCapture(video_path)

# 프레임 수 추출 (진행률 표시용)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 출력 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('tracked_output1.mp4', fourcc, 20.0, (
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
))

# tqdm으로 프레임 루프 진행상황 표시
for _ in tqdm(range(total_frames), desc="처리 중"):
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 탐지
    results = model(frame, verbose=False)[0]

    detections = []
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, int(cls)))

    # Deep SORT 추적
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        center_x = int((l + r) / 2)
        center_y = int((t + b) / 2)

        # 궤적 저장
        trajectories[track_id].append((center_x, center_y))

        # 바운딩 박스 + ID 표시
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 궤적 선 그리기
        pts = trajectories[track_id]
        for j in range(1, len(pts)):
            cv2.line(frame, pts[j - 1], pts[j], (0, 0, 255), 2)

    out.write(frame)

# 종료
cap.release()
out.release()
cv2.destroyAllWindows()

print("완료! 추적된 비디오는 'tracked_output1.mp4'로 저장되었습니다.")


코드 2-3

In [ ]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2

# 1. 모델 및 트래커 초기화
yolo = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=50, n_init=2, nn_budget=100)

# 2. 프레임 반복 처리 (간단 예시)
frame = cv2.imread("scene.jpg")
results = yolo.predict(source=frame, conf=0.25, imgsz=640, verbose=False)[0]

# 3. 탐지 정보 변환
detections = []
for box in results.boxes:
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    conf = box.conf[0].item()
    cls = int(box.cls[0].item())
    detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls, None))

# 4. 추적 결과 처리
tracks = tracker.update_tracks(detections, frame=frame)
for track in tracks:
    if not track.is_confirmed(): continue
    ltrb = track.to_ltrb()
    track_id = track.track_id
    cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0,255,0), 2)
    cv2.putText(frame, f"ID: {track_id}", (int(ltrb[0]), int(ltrb[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

cv2.imshow("Tracking Improved", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()